In [46]:
import pandas as pd
import time
import random

In [43]:
reader=pd.read_csv("../output/reduced_merged_removeT.csv.gz", compression="gzip", chunksize=10000)

In [49]:
def preprocess(x):
    ok=x[x.Response == 1.0]
    x=x[x.Response==0.0]
    size=len(x.index)
    size=int(size/100)
    print(random.sample(x.index, size))
    x=x.loc[random.sample(x.index, size)]
    ret=pd.concat((x,ok), ignore_index=True)
    print(len(ret.index))
    return ret

start=time.time()
df = pd.concat((preprocess(r) for r in reader), ignore_index=True)

TypeError: Population must be a sequence or set.  For dicts, use list(d).